In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
import warnings
from pathlib import Path

import requests


warnings.filterwarnings('error', module='davos')

if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import atexit
import builtins
import os
import shutil
import sys
from os.path import abspath, expandvars

import davos
from davos.core.project import _get_project_name_type

from utils import (
    DavosAssertionError, 
    DavosTestingError, 
    mark, 
    raises, 
    run_tests
)

# tests for `davos.core.project`

In [ ]:
def test_project_registers_atexit_hook():
    """
    When davos was imported and the default Project instance for this 
    notebook was created, a single `cleanup_project_dir_atexit` callback 
    should've been registered with atexit.
    """
    class CompRecorder:
        """
        The list of registered atexit callbacks can't be accessed 
        directly, but in order to *un*register a callback, 
        `atexit.unregister(<fn>)` compares <fn> to each function in the 
        list with `==`, and fails silently if <fn> isn't found. This 
        dummy class records each object it's compared to using `==`, so 
        passing it to `atexit.unregister()` will extrac te list of 
        currently registered functions.
        """
        def __init__(self):
            self.record = []

        def __eq__(self, other):
            self.record.append(other)
            return False
    
    cb_recorder = CompRecorder()
    atexit.unregister(cb_recorder)
    registered_cbs = cb_recorder.record
    target_cb = davos.core.project.cleanup_project_dir_atexit
    # cleanup_project_dir_atexit should be the most recently registered 
    # callback
    assert registered_cbs[-1] is target_cb, (
        f"Expected last registered callback to be {target_cb}, found {registered_cbs[-1]}"
    )
    # there should be only one instance registered at this point
    assert registered_cbs.count(davos.core.project.cleanup_project_dir_atexit) == 1, (
        f"Expected {target_cb} to be registered once. Current atexit "
        f"callbacks:\n{registered_cbs}"
    )

In [ ]:
def test_project_dir_safe_name_convention():
    """
    directory names for notebook-specific projects should be the path 
    to the notebook with the extension removed and path separators 
    replaced with "___"
    """
    nb_path = Path.cwd().joinpath('test_project.ipynb')
    expected_proj_dirname = str(nb_path).rsplit('.ipynb', maxsplit=1)[0].replace(os.sep, '___')
    
    
    # check that name was formatted properly in the object attribute
    assert davos.project.safe_name == expected_proj_dirname, (
        f"Expected project directory to be named {expecrted_proj_dir}. Project "
        f"object's `.safe_name` attribute is {davos.project.project_dir}"
    )
    
    expected_proj_dir = davos.DAVOS_PROJECT_DIR.joinpath(expected_proj_dirname)
    _sep = '\n  - '
    # check that name was formatted properly on the file system
    assert expected_proj_dir.is_dir(), (
        f"{expected_proj_dir} does not exist. Existing project directories are:"
        f"{_sep}{_sep.join([str(d) for d in davos.DAVOS_PROJECT_DIR.iterdir() if d.is_dir()])}"
    )

In [ ]:
def test_unused_project_dirs_removed_shutdown():
    """
    Project directories created while running previous test notebooks
    in which no packages were smuggled should have been removed when the 
    notebook kernel was shutdown.
    Note: test files are collected and run in alphanumeric order
    """
    no_smuggle_notebooks = [
        "test__environment_and_init.ipynb",
        "test_config.ipynb",
        "test_implementations.ipynb",
        "test_ipython_common.ipynb",
        "test_ipython_post7.ipynb",
        "test_ipython_pre7.ipynb",
        "test_parsers.ipynb"
    ]
    dirs_exist = []
    for nb_name in no_smuggle_notebooks:
        safe_format_name = nb_name.rsplit('.ipynb', maxsplit=1)[0].replace(os.sep, '___')
        project_dirpath = davos.DAVOS_PROJECT_DIR.joinpath(safe_format_name)
        if project_dirpath.is_dir():
            dirs_exist.append(project_dirpath)
    
    _sep = '\n  - '
    assert not dirs_exist, (
        "Expected the following directories from previously run test notebooks"
        f"to have been removed on kernel shutdown, but they still exist:{_sep}"
        f"{_sep.join(dirs_exist)}"
    )

In [ ]:
def test_used_project_dirs_persist_shutdown():
    """
    Project directories created while running previous test notebooks
    in which packages *were* smuggled should still exist.
    """
    smuggle_notebooks = ["test_core.ipynb"]
    missing_dirs = []
    for nb_name in smuggle_notebooks:
        nb_path = Path.cwd().joinpath(nb_name)
        safe_format_name = str(nb_path).rsplit('.ipynb', maxsplit=1)[0].replace(os.sep, '___')
        project_dirpath = davos.DAVOS_PROJECT_DIR.joinpath(safe_format_name)
        if not project_dirpath.is_dir():
            missing_dirs.append(str(project_dirpath))
                
    _sep = '\n  - '
    assert not missing_dirs, (
        "Expected the following directories to remain after their associated "
        f"notebook's kernel was shut down, but they do not exist:{_sep}"
        f"{_sep.join(missing_dirs)}"
    )

In [ ]:
def test_get_project_name_type_valid_abs():
    """
    test passing `_get_project_name_type()` an absolute path to an 
    existing notebook file, as both a `str` and a `pathlib.Path`.
    """
    path_input = Path('test_core.ipynb').resolve(strict=True)
    str_input = str(path_input)
    
    expected_proj_name = str_input
    expected_proj_type = davos.core.project.ConcreteProject
    
    # test passing as Path
    proj_name_path, proj_type_path = _get_project_name_type(path_input)
    assert proj_name_path == expected_proj_name, (
        f"Given input:\n\t{path_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name_path!r}"
    )
    assert proj_type_path is expected_proj_type, (
        f"Given input:\n\t{path_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type_path!r}"
    )
    
    # test passing as str
    proj_name_str, proj_type_str = _get_project_name_type(str_input)
    assert proj_name_str == expected_proj_name, (
        f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name_str!r}"
    )
    assert proj_type_str is expected_proj_type, (
        f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type_str!r}"
    )

In [ ]:
def test_get_project_name_type_valid_rel():
    """
    test passing `_get_project_name_type()` a relative path to an 
    existing notebook file in a different directory, as both a `str` and 
    a `pathlib.Path`.
    """
    path_input = Path('../tmp-testing-notebook.ipynb')
    str_input = str(path_input)
    
    expected_proj_name = abspath(str_input)
    expected_proj_type = davos.core.project.ConcreteProject
    
    path_input.touch()
    
    try:
        # test passing as Path
        proj_name_path, proj_type_path = _get_project_name_type(path_input)
        assert proj_name_path == expected_proj_name, (
            f"Given input:\n\t{path_input!r}\nExpected project name to be:\n\t"
            f"{expected_proj_name!r}\nGot:\n\t{proj_name_path!r}"
        )
        assert proj_type_path is expected_proj_type, (
            f"Given input:\n\t{path_input!r}\nExpected project type to be:\n\t"
            f"{expected_proj_type!r}\nGot:\n\t{proj_type_path!r}"
        )

        # test passing as str
        proj_name_str, proj_type_str = _get_project_name_type(str_input)
        assert proj_name_str == expected_proj_name, (
            f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
            f"{expected_proj_name!r}\nGot:\n\t{proj_name_str!r}"
        )
        assert proj_type_str is expected_proj_type, (
            f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
            f"{expected_proj_type!r}\nGot:\n\t{proj_type_str!r}"
        )
    finally:
        # clean up regardless of test outcome
        path_input.unlink()

In [ ]:
def test_get_project_name_type_valid_cwd():
    """
    test passing `_get_project_name_type()` just the name of an existing 
    notebook file in the current working directory, as both a `str` and 
    a `pathlib.Path`.
    """
    path_input = Path('test_core.ipynb')
    str_input = str(path_input)
    
    expected_proj_name = abspath(str_input)
    expected_proj_type = davos.core.project.ConcreteProject
        
    # test passing as Path
    proj_name_path, proj_type_path = _get_project_name_type(path_input)
    assert proj_name_path == expected_proj_name, (
        f"Given input:\n\t{path_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name_path!r}"
    )
    assert proj_type_path is expected_proj_type, (
        f"Given input:\n\t{path_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type_path!r}"
    )

    # test passing as str
    proj_name_str, proj_type_str = _get_project_name_type(str_input)
    assert proj_name_str == expected_proj_name, (
        f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name_str!r}"
    )
    assert proj_type_str is expected_proj_type, (
        f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type_str!r}"
    )

In [ ]:
def test_get_project_name_type_valid_env():
    """
    test passing `_get_project_name_type()` a path to an existing
    notebook file that contains an environment variable, as both a `str` 
    and a `pathlib.Path`.
    """
    path_input = Path('$GITHUB_WORKSPACE/tmp-testing-notebook.ipynb')
    str_input = str(path_input)
    
    expected_proj_name = expandvars(str_input)
    expected_proj_type = davos.core.project.ConcreteProject
    
    Path(expected_proj_name).touch()
    
    try:
        # test passing as Path
        proj_name_path, proj_type_path = _get_project_name_type(path_input)
        assert proj_name_path == expected_proj_name, (
            f"Given input:\n\t{path_input!r}\nExpected project name to be:\n\t"
            f"{expected_proj_name!r}\nGot:\n\t{proj_name_path!r}"
        )
        assert proj_type_path is expected_proj_type, (
            f"Given input:\n\t{path_input!r}\nExpected project type to be:\n\t"
            f"{expected_proj_type!r}\nGot:\n\t{proj_type_path!r}"
        )

        # test passing as str
        proj_name_str, proj_type_str = _get_project_name_type(str_input)
        assert proj_name_str == expected_proj_name, (
            f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
            f"{expected_proj_name!r}\nGot:\n\t{proj_name_str!r}"
        )
        assert proj_type_str is expected_proj_type, (
            f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
            f"{expected_proj_type!r}\nGot:\n\t{proj_type_str!r}"
        )
    finally:
        Path(expected_proj_name).unlink()

In [ ]:
def test_get_project_name_type_valid_tilde():
    """
    test passing `_get_project_name_type()` a path to an existing
    notebook file that contains the ~ (user home directory) construct, 
    as both a `str` and a `pathlib.Path`.
    """
    path_input = Path('~/tmp-testing-notebook.ipynb')
    str_input = str(path_input)
    
    expected_proj_name = str(path_input.expanduser())
    expected_proj_type = davos.core.project.ConcreteProject
    
    Path(expected_proj_name).touch()
    
    try:
        # test passing as Path
        proj_name_path, proj_type_path = _get_project_name_type(path_input)
        assert proj_name_path == expected_proj_name, (
            f"Given input:\n\t{path_input!r}\nExpected project name to be:\n\t"
            f"{expected_proj_name!r}\nGot:\n\t{proj_name_path!r}"
        )
        assert proj_type_path is expected_proj_type, (
            f"Given input:\n\t{path_input!r}\nExpected project type to be:\n\t"
            f"{expected_proj_type!r}\nGot:\n\t{proj_type_path!r}"
        )

        # test passing as str
        proj_name_str, proj_type_str = _get_project_name_type(str_input)
        assert proj_name_str == expected_proj_name, (
            f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
            f"{expected_proj_name!r}\nGot:\n\t{proj_name_str!r}"
        )
        assert proj_type_str is expected_proj_type, (
            f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
            f"{expected_proj_type!r}\nGot:\n\t{proj_type_str!r}"
        )
    finally:
        Path(expected_proj_name).unlink()

In [ ]:
def test_get_project_name_type_valid_abstract():
    """
    test passing `_get_project_name_type()` a theoretically valid path 
    to notebook file that doesn't currently exist (should create an 
    `AbstractProject`), as both a `str` and a `pathlib.Path`.
    """
    path_input = Path('non-existent-notebook.ipynb').resolve(strict=False)
    str_input = str(path_input)
    
    expected_proj_name = str_input
    expected_proj_type = davos.core.project.AbstractProject
    
    # test passing as Path
    proj_name_path, proj_type_path = _get_project_name_type(path_input)
    assert proj_name_path == expected_proj_name, (
        f"Given input:\n\t{path_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name_path!r}"
    )
    assert proj_type_path is expected_proj_type, (
        f"Given input:\n\t{path_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type_path!r}"
    )
    
    # test passing as str
    proj_name_str, proj_type_str = _get_project_name_type(str_input)
    assert proj_name_str == expected_proj_name, (
        f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name_str!r}"
    )
    assert proj_type_str is expected_proj_type, (
        f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type_str!r}"
    )

In [ ]:
def test_get_project_name_type_valid_simple():
    """
    test passing `_get_project_name_type()` a "simple" 
    (non-path/non-notebook-specific) name.
    """
    str_input = 'test-project'
    
    expected_proj_name = str_input
    expected_proj_type = davos.core.project.ConcreteProject

    proj_name, proj_type = _get_project_name_type(str_input)
    assert proj_name == expected_proj_name, (
        f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name!r}"
    )
    assert proj_type is expected_proj_type, (
        f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type!r}"
    )

In [ ]:
def test_get_project_name_type_valid_safe_concrete():
    """
    test passing `_get_project_name_type()` a project name in the "safe"
    format for directory names in `davos.core.project.DAVOS_PROJECT_DIR`
    (non-path/non-notebook-specific) name. The safe-format project name 
    corresponds to an existing notebook file (should create a 
    `ConcreteProject`). 
    """
    # use safe-formatted path to current notebook
    nb_path = Path.cwd().joinpath('test_project.ipynb')
    str_input = str(nb_path).rsplit('.ipynb', maxsplit=1)[0].replace(os.sep, '___')
    
    expected_proj_name = str(nb_path)
    expected_proj_type = davos.core.project.ConcreteProject

    proj_name, proj_type = _get_project_name_type(str_input)
    assert proj_name == expected_proj_name, (
        f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name!r}"
    )
    assert proj_type is expected_proj_type, (
        f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type!r}"
    )

In [ ]:
def test_get_project_name_type_valid_safe_abstract():
    """
    test passing `_get_project_name_type()` a project name in the "safe"
    format for directory names in `davos.core.project.DAVOS_PROJECT_DIR`
    (non-path/non-notebook-specific) name. The safe-format project name 
    corresponds to a notebook file that does not currently exist (should 
    create an `AbstractProject`). 
    """
    str_input = '___path___to___notebook___file'
    
    expected_proj_name = f"{str_input.replace('___', '/')}.ipynb"
    expected_proj_type = davos.core.project.AbstractProject

    proj_name, proj_type = _get_project_name_type(str_input)
    assert proj_name == expected_proj_name, (
        f"Given input:\n\t{str_input!r}\nExpected project name to be:\n\t"
        f"{expected_proj_name!r}\nGot:\n\t{proj_name!r}"
    )
    assert proj_type is expected_proj_type, (
        f"Given input:\n\t{str_input!r}\nExpected project type to be:\n\t"
        f"{expected_proj_type!r}\nGot:\n\t{proj_type!r}"
    )

In [ ]:
def test_get_project_name_type_invalid_names():
    """
    test passing `_get_project_name_type()` various names that 
    would not be valid for a Jupyter notebook file, as both a `str` and 
    a `pathlib.Path`.
    """
    invalid_names = [
        '',                  # denotes CWD (Path('') == Path('.'))
        '.ipynb',            # notebook file names must contain at least one character (check CWD)
        '../.ipynb',         # notebook file names must contain at least one character (check rel path)
        'foo/bar/.ipynb'     # notebook file names must contain at least one character (check abs path)
        'foo:bar.ipynb',     # notebook file names may not contain colons
        'foo\\bar.ipynb',    # notebook file names may not contain backslashes
    ]
    
    didnt_fail = []
    for name in invalid_names:
        # test passing as Path
        try:
            with raises(davos.core.exceptions.DavosProjectError):
                _get_project_name_type(name)
        except DavosAssertionError:
            didnt_fail.append(name)
        # test passing as str
        try:
            with raises(davos.core.exceptions.DavosProjectError):
                _get_project_name_type(Path(name))
        except DavosAssertionError:
            didnt_fail.append(Path(name))
            
    if any(didnt_fail):
        _sep = '\n\t- '
        raise DavosTestingError(
            "The following project names should've been considered invalid "
            f"but weren't:{_sep}{_sep.join(map(repr, didnt_fail))}"
        )

In [ ]:
def test_get_project_name_type_invalid_dirpath():
    """
    test passing `_get_project_name_type()` a path to a  directory 
    (which should fail), as both a `str` and a `pathlib.Path`.
    """
    path_input = Path('tmp-testing-dir').resolve()
    str_input = str(path_input)
    
    path_input.mkdir()
    
    try:
        # test passing as Path
        with raises(davos.core.exceptions.DavosProjectError):
            _get_project_name_type(path_input)
        # test passing as str
        with raises(davos.core.exceptions.DavosProjectError):
            _get_project_name_type(str_input)
    finally:
        path_input.rmdir()

In [ ]:
def test_get_project_name_type_invalid_nonnotebook():
    """
    test passing `_get_project_name_type()` a path to an existing
    non-notebook file (which should fail), as both a `str` and a 
    `pathlib.Path`.
    """
    path_input = Path('tmp-testing-file.txt').resolve()
    str_input = str(path_input)
    
    path_input.touch()
    
    try:
        # test passing as Path
        with raises(davos.core.exceptions.DavosProjectError):
            _get_project_name_type(path_input)
        # test passing as str
        with raises(davos.core.exceptions.DavosProjectError):
            _get_project_name_type(str_input)
    finally:
        path_input.unlink()

In [ ]:
def test_refresh_installed_pkgs():
    """Test `davos.core.project.Project._refresh_installed_pkgs`"""
    # create site-packages dir if it doesn't exist yet
    davos.project.site_packages_dir.mkdir(parents=True)
    
    # call function upfront to set `._site_packages_mtime`
    davos.project._refresh_installed_pkgs()
    initial_site_pkgs_mtime = davos.project._site_packages_mtime
    
    # function shouldn't update stored value if dir hasn't been modified
    davos.project._refresh_installed_pkgs()
    new_site_pkgs_mtime = davos.project._site_packages_mtime
    assert new_site_pkgs_mtime  == initial_site_pkgs_mtime, (
        f'Expected mtime of {davos.project.site_packages_dir} to be '
        f'unchanged, went from {initial_site_pkgs_mtime} to {new_site_pkgs_mtime}'
    )
    
    tmp_filepath = davos.project.site_packages_dir.joinpath('tmp-file.txt')
    tmp_filepath.touch()
    # function should update stored value if dir has been modified
    try:
        davos.project._refresh_installed_pkgs()
        new_site_pkgs_mtime = davos.project._site_packages_mtime
        assert new_site_pkgs_mtime  != initial_site_pkgs_mtime, (
            f'Expected mtime of {davos.project.site_packages_dir} to be '
            f'updated, but no change (mtime: {initial_site_pkgs_mtime})'
        )
    finally:
        tmp_filepath.unlink()

In [ ]:
@mark.skipif(sys.version_info < (3, 7), reason='Runs on Python >= 3.7')
def test_installed_packages():
    """Test `davos.core.project.Project.installed_packages`"""
    initial_installed_pkgs = davos.project.installed_packages
    _sep = '\n'
    assert not any(initial_installed_pkgs), (
        'Expected no packages to be initially installed in project. Found:\n'
        f'{_sep.join(initial_installed_pkgs)}'
    )
    # v4.64.1 was the last version to support Python 3.6
    # pass --ignore-installed to ensure packge gets installed in project 
    # rather than using version in notebook kernel environment, just in 
    # case the exact requested version is installed there
    smuggle tqdm    # pip: tqdm==4.64.1 --ignore-installed
    new_installed_pkgs = davos.project.installed_packages
    assert new_installed_pkgs == [('tqdm', '4.64.1')], (
        "Expected installed_packages to be:\n[('tqdm', '4.64.1')]\nFound:\n"
        f"{new_installed_pkgs}"
    )

In [ ]:
@mark.skipif(sys.version_info < (3, 7), reason='Runs on Python >= 3.7')
def test_freeze():
    """Test `davos.core.project.Project.freeze`"""
    # smuggle exact versions of two projects with no dependencies so we 
    # know exactly what the output should look like
    smuggle tqdm    # pip: tqdm==4.64.1
    smuggle fastdtw    # pip: fastdtw==0.3.2
    
    expected_output = 'fastdtw==0.3.2\ntqdm==4.64.1'
    output = davos.project.freeze()
    
    assert output == expected_output, (
        f'Expected output:\n{expected_output}\n=====Observed output:\n{output}'
    )

In [ ]:
@mark.timeout(30)
def test_remove_current_project():
    """
    Test `davos.core.project.Project.remove` on the currently active 
    project. Should remove all packages but leave the project directory.
    """
    # create & switch to temporary project (copy of main project) for test
    initial_project = davos.project
    tmp_proj_name = 'tmp-project'
    tmp_proj_dirpath = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name)
    shutil.copytree(davos.project.project_dir, tmp_proj_dirpath)
    try:
        davos.project = davos.core.project.ConcreteProject(tmp_proj_name)
        assert davos.project.project_dir == tmp_proj_dirpath
        
        davos.project.remove(yes=True)
        
        assert tmp_proj_dirpath.is_dir(), (
            f'Expected project directory ({tmp_proj_dirpath}) to still exist '
            'after removal (currently active project)'
        )
        
        assert not any(tmp_proj_dirpath.iterdir()), (
            f'Expected project directory to be empty after removal but found '
            f'the following items:\n{list(tmp_proj_dirpath.iterdir())}'
        )
    finally:
        davos.project = initial_project
        if tmp_proj_dirpath.is_dir():
            shutil.rmtree(tmp_proj_dirpath)

In [ ]:
@mark.timeout(30)
def test_remove_noncurrent_project():
    """
    Test `davos.core.project.Project.remove` on a project that is not 
    currently in use. Should remove all packages and the project 
    directory.
    """
    # create temporary project directory (copy of main project) for test
    tmp_proj_name = 'tmp-project'
    tmp_proj_dirpath = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name)
    shutil.copytree(davos.project.project_dir, tmp_proj_dirpath)
    
    tmp_project = davos.core.project.ConcreteProject(tmp_proj_name)
    
    try:
        tmp_project.remove(yes=True)
        assert not tmp_proj_dirpath.is_dir(), (
            f'Expected project directory ({tmp_proj_dirpath}) not to exist '
            'after removal (not currently active project)'
        )
    except:
        shutil.rmtree(tmp_proj_dirpath)
        raise

In [ ]:
@mark.timeout(30)
def test_remove_fails_noconfirm_noninteractive():
    """
    Test that `davos.core.project.Project.remove` fails when 
    non-interactive mode is enabled and `yes=True` is not explicitly 
    passed.
    """
    # test on a temporary random project in case .remove()` somehow 
    # succeeds (test fails)
    tmp_project = davos.core.project.ConcreteProject('tmp-project')
    # get project dir to remove after test passes
    tmp_project_dir = tmp_project.project_dir
    
    davos.config.noninteractive = True
    try:
        with raises(davos.core.exceptions.DavosProjectError):
            tmp_project.remove()
    except DavosAssertionError:
        raise
    else:
        # if test passes (fails to remove project), clean up project dir
        tmp_project_dir.rmdir()
    finally:
        davos.config.noninteractive = False

In [ ]:
@mark.timeout(30)
def test_remove_user_cancels():
    """
    Test that `davos.core.project.Project.remove` takes no action if
    the user declines to remove the project when prompted for 
    confirmation.
    """
    def _mock_input_no(prompt=None):
        # mock user input of "n" (for "no")
        return 'n'
    
    
    # test on a temporary random project in case .remove()` somehow 
    # succeeds (test fails)
    tmp_project = davos.core.project.ConcreteProject('tmp-project')
    tmp_project_dir = tmp_project.project_dir
    # make sure the project directory was created 
    assert tmp_project_dir.is_dir(), (
        f'Expected {tmp_project_dir} to exist at start of test'
    )
    
    davos.core.core.input = _mock_input_no
    try:
        tmp_project.remove()
        assert tmp_project_dir.is_dir(), (
            f'Expected {tmp_project_dir} to exist after responding "n" to '
            'user confirmation prompt.'
        )
    except:
        raise
    else:
        tmp_project_dir.rmdir()
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_rename_updates_dirname():
    """
    Test that `davos.core.project.Project.rename` updates the name of 
    the project directory in `davos.core.project.DAVOS_PROJECT_DIR`.
    
    Initial and new project names are both "simple" names.
    """
    # create temporary project directory (copy of main project) for test
    tmp_proj_name_initial = 'tmp-project-initial'
    tmp_proj_dirpath_initial = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name_initial)
    shutil.copytree(davos.project.project_dir, tmp_proj_dirpath_initial)
    tmp_project = davos.core.project.ConcreteProject(tmp_proj_name_initial)
    
    tmp_proj_name_new = 'tmp-project-new'
    tmp_proj_dirpath_new = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name_new)
    
    try:
        tmp_project.rename(tmp_proj_name_new)
        assert tmp_proj_dirpath_new.is_dir(), (
            f'Expected directory with new project name ({tmp_proj_name_new}) '
            'to exist after renaming project'
        )
        assert not tmp_proj_dirpath_initial.is_dir(), (
            'Expected directory with initial project name '
            f'({tmp_proj_name_new}) to not exist after renaming project'
        )
    finally:
        if tmp_proj_dirpath_new.is_dir():
            shutil.rmtree(tmp_proj_dirpath_new)
        if tmp_proj_dirpath_initial.is_dir():
            shutil.rmtree(tmp_proj_dirpath_initial)

In [ ]:
def test_rename_updates_attrs():
    """
    Test that `davos.core.project.Project.rename` updates all relevant 
    attributes of the existing Project object with the new name. Also 
    check that the renamed project is still a 
    `davos.core.project.ConcreteProject`.
    
    Initial and new project names are both "simple" names.
    """
    # create temporary project directory (copy of main project) for test
    tmp_proj_name_initial = 'tmp-project-initial'
    tmp_proj_dirpath_initial = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name_initial)
    shutil.copytree(davos.project.project_dir, tmp_proj_dirpath_initial)
    tmp_project = davos.core.project.ConcreteProject(tmp_proj_name_initial)
    
    tmp_proj_name_new = 'tmp-project-new'
    tmp_proj_dirpath_new = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name_new)
    
    expected_attrs_values = [
        ('name', tmp_proj_name_new),
        ('safe_name', tmp_proj_name_new),
        ('project_dir', tmp_proj_dirpath_new),
        ('site_packages_dir', tmp_proj_dirpath_new.joinpath(
            'lib', 
            f'python{sys.version_info.major}.{sys.version_info.minor}', 
            'site-packages'
        ))
    ]
    
    try:
        tmp_project.rename(tmp_proj_name_new)
        
        for (attr, expected_val) in expected_attrs_values:
            observed_val = getattr(tmp_project, attr)
            assert observed_val == expected_val, (
                f'Expected value of tmp_project.{attr} to be:\n\t'
                f'{expected_val}\nValue is:\n\t{observed_val}'
            )
    finally:
        if tmp_proj_dirpath_new.is_dir():
            shutil.rmtree(tmp_proj_dirpath_new)
        if tmp_proj_dirpath_initial.is_dir():
            shutil.rmtree(tmp_proj_dirpath_initial)

In [ ]:
def test_rename_concrete_abstract():
    """
    Test renaming a project with a notebook-specific/path-based name 
    from an existing notebook path to one that doesn't exist. Project 
    object should be converted from a 
    `davos.core.project.ConcreteProject` to a 
    `davos.core.project.AbstractProject`.
    """
    tmp_notebook_name_initial = 'tmp-notebook-exists.ipynb'
    tmp_notebook_name_new = 'tmp-notebook-nonexistent.ipynb'
    
    # set up temporary resources for test:
    
    # - notebook file
    tmp_notebook_path_initial = Path.cwd().joinpath(tmp_notebook_name_initial)
    tmp_notebook_path_new = tmp_notebook_path_initial.with_name(tmp_notebook_name_new)
    tmp_notebook_path_initial.touch()
    
    # - project directory (copy of main project)
    tmp_project_safename_initial = (str(tmp_notebook_path_initial)
                                    .rsplit('.ipynb', maxsplit=1)[0]
                                    .replace(os.sep, '___'))
    tmp_project_dirpath_initial = davos.DAVOS_PROJECT_DIR.joinpath(tmp_project_safename_initial)
    shutil.copytree(davos.project.project_dir, tmp_project_dirpath_initial)
    
    # - project object
    tmp_project = davos.core.project.ConcreteProject(tmp_notebook_path_initial)
    
    try:
        tmp_project.rename(tmp_notebook_name_new)
        assert isinstance(tmp_project, davos.core.project.AbstractProject), (
            'Expected project object type to be '
            f'{davos.core.project.AbstractProject} after renaming. Type is '
            f'{type(tmp_project)}'
        )
    finally:
        # clean up temporary resources
        tmp_project_safename_new = (str(tmp_notebook_path_new)
                                    .rsplit('.ipynb', maxsplit=1)[0]
                                    .replace(os.sep, '___'))
        tmp_project_dirpath_new = tmp_project_dirpath_initial.with_name(tmp_project_safename_new)
        if tmp_project_dirpath_new.is_dir():
            shutil.rmtree(tmp_project_dirpath_new)
        if tmp_project_dirpath_initial.is_dir():
            shutil.rmtree(tmp_project_dirpath_initial)
        tmp_notebook_path_initial.unlink()

In [ ]:
def test_rename_abstract_concrete():
    """
    Test renaming a project with a notebook-specific/path-based name 
    from a notebook path that doesn't exist to one that does. Project 
    object should be converted from a 
    `davos.core.project.AbstractProject` to a 
    `davos.core.project.ConcreteProject`.
    """
    tmp_notebook_name_initial = 'tmp-notebook-nonexistent.ipynb'
    tmp_notebook_name_new = 'tmp-notebook-exists.ipynb'
    
    # set up temporary resources for test:
    
    # - notebook file
    tmp_notebook_path_initial = Path.cwd().joinpath(tmp_notebook_name_initial)
    tmp_notebook_path_new = tmp_notebook_path_initial.with_name(tmp_notebook_name_new)
    tmp_notebook_path_new.touch()
    
    # - project directory (copy of main project)
    tmp_project_safename_initial = (str(tmp_notebook_path_initial)
                                    .rsplit('.ipynb', maxsplit=1)[0]
                                    .replace(os.sep, '___'))
    tmp_project_dirpath_initial = davos.DAVOS_PROJECT_DIR.joinpath(tmp_project_safename_initial)
    shutil.copytree(davos.project.project_dir, tmp_project_dirpath_initial)
    
    # - project object
    tmp_project = davos.core.project.AbstractProject(tmp_notebook_path_initial)
    
    try:
        tmp_project.rename(tmp_notebook_name_new)
        assert isinstance(tmp_project, davos.core.project.ConcreteProject), (
            'Expected project object type to be '
            f'{davos.core.project.ConcreteProject} after renaming. Type is '
            f'{type(tmp_project)}'
        )
    finally:
        # clean up temporary resources
        tmp_project_safename_new = (str(tmp_notebook_path_new)
                                    .rsplit('.ipynb', maxsplit=1)[0]
                                    .replace(os.sep, '___'))
        tmp_project_dirpath_new = tmp_project_dirpath_initial.with_name(tmp_project_safename_new)
        if tmp_project_dirpath_new.is_dir():
            shutil.rmtree(tmp_project_dirpath_new)
        if tmp_project_dirpath_initial.is_dir():
            shutil.rmtree(tmp_project_dirpath_initial)
        tmp_notebook_path_new.unlink()

In [ ]:
def test_rename_fails_name_inuse():
    """
    Test that `davos.core.project.Project.rename` fails if the specified 
    new name is already being used by another project in which any 
    packages have been installed.
    
    Initial and new project names are both "simple" names.
    """
    # create temporary project directory (copy of main project) for test
    tmp_proj_name_initial = 'tmp-project-initial'
    tmp_proj_dirpath_initial = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name_initial)
    shutil.copytree(davos.project.project_dir, tmp_proj_dirpath_initial)
    
    tmp_proj_name_new = 'tmp-project-new'
    tmp_proj_dirpath_new = davos.DAVOS_PROJECT_DIR.joinpath(tmp_proj_name_new)
    shutil.copytree(davos.project.project_dir, tmp_proj_dirpath_new)
    
    tmp_project = davos.core.project.ConcreteProject(tmp_proj_name_initial)
    
    try:
        with raises(davos.core.exceptions.DavosProjectError):
            tmp_project.rename(tmp_proj_name_new)
    finally:
        if tmp_proj_dirpath_new.is_dir():
            shutil.rmtree(tmp_proj_dirpath_new)
        if tmp_proj_dirpath_initial.is_dir():
            shutil.rmtree(tmp_proj_dirpath_initial)

In [ ]:
def test_dir_is_empty():
    """Test `davos.core.project._dir_is_empty`"""
    test_dir = Path('tmp-testing-dir').resolve()
    test_dir.mkdir()
    try:
        assert davos.core.project._dir_is_empty(test_dir)
        test_dir.joinpath('.DS_Store').touch()
        assert davos.core.project._dir_is_empty(test_dir)
        test_dir.joinpath('foo.txt').touch()
        assert not davos.core.project._dir_is_empty(test_dir)
    finally:
        shutil.rmtree(test_dir)

In [ ]:
def test_get_notebook_path():
    """Test `davos.core.project.get_notebook_path`"""
    expected_nbpath = str(Path.cwd().joinpath('test_project.ipynb'))
    computed_nbpath = davos.core.project.get_notebook_path()
    assert computed_nbpath == expected_nbpath, (
        '`davos.core.project.get_notebook_path()` returned an incorrect path '
        f'for the current notebook.Expected:\n\t{expected_nbpath}\nGot:'
        f'{computed_nbpath}'
    )

In [ ]:
def test_use_default_project():
    """Test `davos.core.project.use_default_project`"""
    curr_notebook_path = Path.cwd().joinpath('test_project.ipynb')
    expected_default_project = davos.core.project.ConcreteProject(curr_notebook_path)
    
    try:
        # should start the test using the default project
        assert davos.project == expected_default_project, (
            'Expected the initially active project to be the default project:\n\t'
            f'{expected_default_project}\nActive project is:\n\t{davos.project}'
        )
        
        # temporarily switch to a different project
        davos.project = davos.core.project.ConcreteProject('tmp-project')
        assert davos.project != expected_default_project, (
            'Expected the active project to no longer be the default'
        )
        
        # switch back to the default with `use_default_project()`
        davos.use_default_project()
        assert davos.project == expected_default_project, (
            'Expected the active project to be the default project:\n\t'
            f'{expected_default_project}\nActive project is:\n\t{davos.project}'
        )
    finally:
        davos.project = expected_default_project

In [ ]:
run_tests()